In [1]:
%%time
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 

from binance import AsyncClient, BinanceSocketManager, Client, ThreadedWebsocketManager
import Artha.configs.binance_config as c 
key = c.apis[0][0]
secret = c.apis[0][1]
client = Client(key, secret)

import asyncio
import time
import nest_asyncio
nest_asyncio.apply()

# client.get_historical_klines("ETHBTC", "1h", "1 Dec, 2017", "1 Jan, 2018")
import pandas as pd
pd.options.display.float_format = '{:.9f}'.format
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# from Artha.mentions import *
from datetime import datetime, timedelta
import numpy as np

from Artha.trading import *

import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot

CPU times: user 1.15 s, sys: 283 ms, total: 1.43 s
Wall time: 2.43 s


In [2]:
%%time
ticker, time_frame = "ETHBTC", "4h"
klines = get_klines(ticker, time_frame, "01/01/2021 00:00:00")
df = get_klines_df(klines)
ichimoku(df)

TypeError: ichimoku() missing 1 required positional argument: 'ichi'

In [ ]:

# a, b, c, d, e = df["Close"], df["Low"], df['cloud_top'], df['bull_kumo_pricecross'], df['bull_kumo_pricecross1']
# for j in range(len(df.values)):
#     # if d[j] == 1:
#     #     for i in range(j, j+2):
#     # # print(i, a[i], b[i], (a[i]+b[i])/2, c[i], d[i], e[i])
#     #         print(a[i], c[i], d[i])
#     #     print("\n")
#     if  e[j] ==1:
#         print("true")
#         print(a[j], a[j+1], c[j])
#     #     for i in range(j, j+2):
#     # # print(i, a[i], b[i], (a[i]+b[i])/2, c[i], d[i], e[i])
#     #         print(a[i], c[i], d[i])
#         print("next\n")

In [ ]:
%%time

d = df 
fig = setup_ichi_graph(d, ticker, time_frame)

        
iplot( fig, filename = 'candlestick-ichimoku' )

In [ ]:
markets = get_quote_dict()
btc_markets = [tick+asset for asset in list(markets.keys())[:1] for tick in markets[asset]]

In [ ]:
# downloading data

%%time
import concurrent.futures
start_date, window_count = "03/01/2021 00:00:00", 63
def save_klines(asset):
    klines = get_klines(asset, "1h", start_date)
    df = get_klines_df(klines)
    df.to_csv("../data/crypto_price_data/"+asset+".csv", index=True)
    return "done "+asset

def all_klines(all_assets):
    with concurrent.futures.ThreadPoolExecutor(max_workers = 60) as executor:
        result = [executor.submit(save_klines, asset) for asset in all_assets]
    for future in concurrent.futures.as_completed(result):
        print(future.result())
    
# all_klines(np.setdiff1d(btc_markets[:30], dd))
# dd = [i[:-4] for i in os.listdir("../data/crypto_price_data/")]

# np.setdiff1d(dd, btc_markets[:30])
save_klines("MATICBTC")

In [29]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[1, 2, 3, 4], y=[0, 2, 3, 5])) # fill down to xaxis
fig.add_trace(go.Scatter(x=[1, 2, 3, 4], y=[3, 5, 1, 7], fill='tonexty')) # fill to trace0 y

fig.show()

In [ ]:
# saving data
files = ["../data/crypto_price_data/"+i for i in os.listdir("../data/crypto_price_data")]
file_num = 17

df = pd.read_csv(files[file_num])
df['Date'] = pd.to_datetime(df['Unnamed: 0'])
df = df.set_index("Date").drop(['Unnamed: 0'], axis = 1)
ticker = os.listdir("../data/crypto_price_data")[file_num][:-4]
print(ticker)

In [ ]:
# writing tests for vol spikes
with open("../tests/test_data/trading_data.py", "w+") as w:
    w.write("volume_data = [\n")
    w.write("("+str(list(df["Volume"].iloc[1640:1648])) + ", "+str([6, 7]) + ", "+str(2)+"),\n")
    w.write("("+str(list(df["Volume"].iloc[253:259])) + ", "+str([3]) + ", "+str(2)+"),\n")
    w.write("]")

In [ ]:
async def single(name):
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client, user_timeout = 20)
    # start any sockets here, i.e a trade socket
    ts = bm.kline_socket(name, interval="5m")
    # then start receiving messages
    async with ts as tscm:
        while True:
            res = await tscm.recv()
            print(res)
   
    await client.close_connection()

async def multiple (asset_names):
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client, user_timeout = 20)
    # start any sockets here, i.e a trade socket
    ms = bm.multiplex_socket(asset_names)
    # then start receiving messages
    async with ms as tscm:
        while True:
            res = await tscm.recv()
            print(res["data"]["k"])
   
    await client.close_connection()

In [ ]:
import Artha.crypto_data as crypto
markets = crypto.get_market_dict()
print("BTC" in [markets.keys()])
print("BTC" in [i for i in crypto.get_invert_dict(markets).keys()])
s_type = "@kline_5m"
btc_markets = [(tick+asset+s_type).lower() for asset in list(markets.keys())[:1] for tick in markets[asset]]

In [ ]:
loop = asyncio.get_event_loop()
loop.run_until_complete(multiple(btc_markets[:3]))
print("done")

In [ ]:
symbol = 'BNBBTC'

twm = ThreadedWebsocketManager(api_key=key, api_secret=secret)
# start is required to initialise its internal loop
twm.start()

def handle_socket_message(msg):
    print(f"message type: {msg['e']}")
    print(msg)

twm.start_kline_socket(callback=handle_socket_message, symbol=symbol)

# multiple sockets can be started
twm.start_depth_socket(callback=handle_socket_message, symbol=symbol)

# or a multiplex socket can be started like this
# see Binance docs for stream names
streams = ['BNBBTC@miniTicker', 'BNBBTC@bookTicker']
twm.start_multiplex_socket(callback=handle_socket_message, streams=streams)

In [ ]:
start_date, window_count = "03/01/2021 00:00:00", 63

dates = [from_datetime(to_datetime(start_date)+timedelta(days=i)) for i in range(window_count)]

pdrank = pd.read_csv("pgrankdata.csv")
pdrank['Date'] = pd.to_datetime(pdrank['Unnamed: 0'])
pdrank = pdrank.set_index("Date").drop(['Unnamed: 0'], axis = 1)

i = 0
ticker = "DOGE"

while i<len(dates)-1:
    if pdrank[ticker][i]!=0:
        di = pdrank[ticker][i+1]/pdrank[ticker][i]
        if di> 1.1:
            print(i, di, pdrank[ticker][i+1], pdrank[ticker][i])
            print(pdrank[ticker].index[i+1])
    else:
        if pdrank[ticker][i+1] > 0:
            print("new", i, pdrank[ticker][i+1])
    i+=1

In [ ]:
# direct from neo

inc = 32
wind = 30

if ticker == "BTC":
    asset = ticker+"USDT"
else:
    asset = ticker+"BTC"

interval = '2h'
newest, oldest = pdrank[ticker].index[inc-wind: inc+wind][-1].to_pydatetime().timestamp() * 1000, pdrank[ticker].index[inc-wind: inc+wind][0].to_pydatetime().timestamp() * 1000, 
df = crypto.get_klines_df(asset, interval, str(oldest), str(newest))
df.keys()


plt.rcParams['figure.figsize'] = [10, 5]
ts = df["Volume"].astype(float) #.pct_change()

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()


ax1.plot(pdrank[ticker].index[inc-wind: inc+wind], pdrank[ticker][inc-wind: inc+wind], color='b')

ax2.plot(ts.index, ts, color='g')

ax1.set_xlabel('Time')
ax1.set_ylabel(ticker+" PageRank Score", color='b')
ax2.set_ylabel(ticker +" Price", color='g')
# plt.plot([inc]*2, range(-1,1))
plt.title(asset+" "+from_datetime(pdrank[ticker].index[inc].to_pydatetime())[:-9])
plt.gcf().autofmt_xdate()